In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("rakuten_text_train_v1.csv")
df.head()

,productid,imageid,prdtypecode,designation_cleaned,description_cleaned,text_cleaned,dup_count,is_duplicated_group
0,3804725264,1263597046,10,olivia: personalisiertes notizbuch 150 seiten ...,NaN,olivia: personalisiertes notizbuch 150 seiten ...,1,False
1,436067568,1008141237,2280,journal arts (le) n° 133 28/09/2001 l'art marc...,NaN,journal arts (le) n° 133 28/09/2001 l'art marc...,1,False
2,201115110,938777978,50,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,grand stylet ergonomique bleu gamepad nintendo...,1,False
3,50418756,457047496,1280,peluche donald europe disneyland 2000 (marionn...,NaN,peluche donald europe disneyland 2000 (marionn...,1,False
4,278535884,1077757786,2705,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,guerre tuques luc idées grandeur veut organise...,1,False


# ============================================
# 2. Fonctions utilitaires pour les features
# ============================================

In [ ]:
# Toutes les colonnes texte utilisées par TF-IDF doivent être des chaînes non nulles
text_cols_base = ["designation_cleaned", "description_cleaned", "text_cleaned"]
for col in text_cols_base:
    if col in df.columns:
        df[col] = df[col].fillna("").astype(str)

# S'assurer que les colonnes de duplication sont numériques
for col in ["dup_count", "is_duplicated_group"]:
    if col in df.columns:
        # Conversion robuste en numérique, puis remplacement des NaN par 0
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

print("Forme du DataFrame :", df.shape)
print(df[text_cols_base].head())

def safe_str(s):
    """Convertit une valeur en chaîne de caractères sûre (sans NaN)."""
    if isinstance(s, str):
        return s
    if pd.isna(s):
        return ""
    return str(s)

Forme du DataFrame : (84916, 8)
                                 designation_cleaned  \
0  olivia: personalisiertes notizbuch 150 seiten ...   
1  journal arts (le) n° 133 28/09/2001 l'art marc...   
2  grand stylet ergonomique bleu gamepad nintendo...   
3  peluche donald europe disneyland 2000 (marionn...   
4                                      guerre tuques   

                                 description_cleaned  \
0                                                      
1                                                      
2  pilot style touch pen marque speedlink stylet ...   
3                                                      
4  luc idées grandeur veut organiser jeu guerre b...   

                                        text_cleaned  
0  olivia: personalisiertes notizbuch 150 seiten ...  
1  journal arts (le) n° 133 28/09/2001 l'art marc...  
2  grand stylet ergonomique bleu gamepad nintendo...  
3  peluche donald europe disneyland 2000 (marionn...  
4  guerre tuques lu

# ---------- 2.1 Statistiques de base sur le texte ----------

In [4]:
def basic_text_stats(s: str) -> dict:
    """Calcule des statistiques simples sur une chaîne de caractères."""
    s = safe_str(s)
    s_strip = s.strip()
    
    length_char = len(s_strip)
    tokens = s_strip.split()
    length_tokens = len(tokens)

    num_digits = sum(ch.isdigit() for ch in s_strip)
    num_upper = sum(ch.isupper() for ch in s_strip)
    num_punct = sum(ch in ".,;:!?/\\-+()" for ch in s_strip)

    ratio_digits = num_digits / length_char if length_char > 0 else 0.0
    ratio_upper = num_upper / length_char if length_char > 0 else 0.0

    size_pattern = re.compile(r"\b\d+\s*(cm|mm|ml|l|kg|g)\b", flags=re.IGNORECASE)
    num_size = len(size_pattern.findall(s_strip))

    return {
        "len_char": length_char,
        "len_tokens": length_tokens,
        "ratio_digits": ratio_digits,
        "ratio_upper": ratio_upper,
        "num_punct": num_punct,
        "num_size_patterns": num_size,
    }

# ---------- 2.2 Features "pragmatiques" sur le texte ----------

In [5]:
EXCLAMATION_RE = re.compile(r"!")
QUESTION_RE = re.compile(r"\?")
PLUS_RE = re.compile(r"\+")
SLASH_RE = re.compile(r"/")

# Petites listes de mots pour le matériau et la couleur
MATERIAL_WORDS = {"bois", "metal", "métal", "acier", "coton", "plastique", "cuir"}
COLOR_WORDS = {"rouge", "bleu", "noir", "blanc", "rose", "gris", "vert", "jaune"}

def meta_features_from_text(s: str) -> dict:
    """Extrait des features "pragmatiques" d'un texte nettoyé."""
    s = safe_str(s)
    s_lower = s.lower()
    tokens = s_lower.split()
    
    return {
        "num_exclamation": len(EXCLAMATION_RE.findall(s)),
        "num_question": len(QUESTION_RE.findall(s)),
        "num_plus": len(PLUS_RE.findall(s)),
        "num_slash": len(SLASH_RE.findall(s)),
        "num_material_words": sum(word in MATERIAL_WORDS for word in tokens),
        "num_color_words": sum(word in COLOR_WORDS for word in tokens),
    }

# ---------- 2.3 Features "produit" simples ----------

In [6]:
def product_level_features(df: pd.DataFrame) -> pd.DataFrame:
    """Crée des features simples au niveau produit (description présente, longueur, etc.)."""
    # Ici, on n'a plus la colonne 'description' brute, donc on
    # utilise 'description_cleaned' comme approximation.
    df["has_description"] = df["description_cleaned"].fillna("").apply(
        lambda s: int(len(safe_str(s).strip()) > 0)
    )

    df["desc_len_tokens_raw"] = df["description_cleaned"].fillna("").apply(
        lambda s: len(safe_str(s).split())
    )

    # Seuil arbitraire, à ajuster après EDA si besoin
    df["desc_is_long"] = (df["desc_len_tokens_raw"] > 50).astype(int)

    df["title_has_digit"] = df["designation_cleaned"].fillna("").apply(
        lambda s: int(any(ch.isdigit() for ch in safe_str(s)))
    )

    return df

# ============================================
# 3. Application des features de structure
# ============================================

In [7]:
# --- 3.1 Statistiques de base sur titre et description ---
for col in ["designation_cleaned", "description_cleaned"]:
    stats_series = df[col].apply(basic_text_stats)
    stats_df = pd.DataFrame(list(stats_series))
    for stat_col in stats_df.columns:
        new_col_name = f"{col}_{stat_col}"
        df[new_col_name] = stats_df[stat_col]

# --- 3.2 Features pragmatiques sur text_cleaned ---
meta_series = df["text_cleaned"].apply(meta_features_from_text)
meta_df = pd.DataFrame(list(meta_series))
for c in meta_df.columns:
    df[c] = meta_df[c]

# --- 3.3 Features produit ---
df = product_level_features(df)

print("Colonnes après ajout des features :")
print(df.columns)

Colonnes après ajout des features :
Index(['productid', 'imageid', 'prdtypecode', 'designation_cleaned',
       'description_cleaned', 'text_cleaned', 'dup_count',
       'is_duplicated_group', 'designation_cleaned_len_char',
       'designation_cleaned_len_tokens', 'designation_cleaned_ratio_digits',
       'designation_cleaned_ratio_upper', 'designation_cleaned_num_punct',
       'designation_cleaned_num_size_patterns', 'description_cleaned_len_char',
       'description_cleaned_len_tokens', 'description_cleaned_ratio_digits',
       'description_cleaned_ratio_upper', 'description_cleaned_num_punct',
       'description_cleaned_num_size_patterns', 'num_exclamation',
       'num_question', 'num_plus', 'num_slash', 'num_material_words',
       'num_color_words', 'has_description', 'desc_len_tokens_raw',
       'desc_is_long', 'title_has_digit'],
      dtype='object')


# ============================================
# 4. Préparation des listes de features
# ============================================

In [8]:
# Colonnes textuelles pour TF-IDF (titre et description séparés)
text_cols = ["designation_cleaned", "description_cleaned"]

# Colonnes numériques (meta + stats) à inclure dans le modèle
meta_cols = [
    # Features produit
    "has_description",
    "desc_len_tokens_raw",
    "desc_is_long",
    "title_has_digit",
    
    # Statistiques de base sur le titre
    "designation_cleaned_len_char",
    "designation_cleaned_len_tokens",
    "designation_cleaned_ratio_digits",
    "designation_cleaned_ratio_upper",
    "designation_cleaned_num_punct",
    "designation_cleaned_num_size_patterns",
    
    # Statistiques de base sur la description
    "description_cleaned_len_char",
    "description_cleaned_len_tokens",
    "description_cleaned_ratio_digits",
    "description_cleaned_ratio_upper",
    "description_cleaned_num_punct",
    "description_cleaned_num_size_patterns",
    
    # Features pragmatiques
    "num_exclamation",
    "num_question",
    "num_plus",
    "num_slash",
    "num_material_words",
    "num_color_words",
    
    # Features de duplication (issues du notebook 1)
    "dup_count",
    "is_duplicated_group",
]

# On s'assure que toutes les colonnes existent
meta_cols = [c for c in meta_cols if c in df.columns]

print("Nombre de colonnes numériques sélectionnées :", len(meta_cols))

# Variable cible
y = df["prdtypecode"].values

# DataFrame de travail pour le ColumnTransformer
X = df[text_cols + meta_cols]

print("Forme de X :", X.shape)

Nombre de colonnes numériques sélectionnées : 24
Forme de X : (84916, 26)


# ============================================
# 5. Construction du ColumnTransformer + Pipeline
# ============================================

In [9]:
# Vectoriseur TF-IDF pour le titre
tfidf_title = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.8,
    lowercase=False,
    tokenizer=str.split,  # Le texte est déjà nettoyé et séparé par espaces
)

# Vectoriseur TF-IDF pour la description
tfidf_desc = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.8,
    lowercase=False,
    tokenizer=str.split,
)

# Standardisation des features numériques
num_scaler = StandardScaler(with_mean=False)  # with_mean=False pour compatibilité avec les matrices creuses

# ColumnTransformer qui combine texte + numérique
preprocess = ColumnTransformer(
    transformers=[
        ("title_tfidf", tfidf_title, "designation_cleaned"),
        ("desc_tfidf", tfidf_desc, "description_cleaned"),
        ("numeric", num_scaler, meta_cols),
    ],
    remainder="drop",        # On ne garde que ce qui est spécifié
    sparse_threshold=0.3,    # On garde une sortie creuse si possible
)

# Modèle linéaire pour la classification multi-classe
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",  # Important pour les classes déséquilibrées
    solver="saga",
    n_jobs=-1,
)

# Pipeline complet : prétraitement + modèle
clf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", log_reg),
    ]
)

print(clf_pipeline)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('title_tfidf',
                                                  TfidfVectorizer(lowercase=False,
                                                                  max_df=0.8,
                                                                  max_features=20000,
                                                                  min_df=5,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<method 'split' of 'str' objects>),
                                                  'designation_cleaned'),
                                                 ('desc_tfidf',
                                                  TfidfVectorizer(lowercase=False,
                                                                  max_df=0.8,
     

# ============================================
# 6. Split entraînement / validation et apprentissage
# ============================================

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Taille X_train :", X_train.shape)
print("Taille X_valid :", X_valid.shape)

# Entraînement du pipeline complet
print("Entraînement du modèle (pipeline complet)...")
clf_pipeline.fit(X_train, y_train)

# Prédiction sur le jeu de validation
y_pred = clf_pipeline.predict(X_valid)

# Évaluation
weighted_f1 = f1_score(y_valid, y_pred, average="weighted")
print(f"Weighted F1 (validation) : {weighted_f1:.4f}")

print("\nClassification report :")
print(classification_report(y_valid, y_pred))

Taille X_train : (67932, 26)
Taille X_valid : (16984, 26)
Entraînement du modèle (pipeline complet)...


/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Weighted F1 (validation) : 0.7417

Classification report :
              precision    recall  f1-score   support

          10       0.32      0.63      0.42       623
          40       0.80      0.50      0.62       502
          50       0.76      0.72      0.74       336
          60       0.91      0.73      0.81       166
        1140       0.71      0.76      0.74       534
        1160       0.70      0.89      0.79       791
        1180       0.50      0.58      0.54       153
        1280       0.75      0.48      0.58       974
        1281       0.52      0.56      0.54       414
        1300       0.83      0.84      0.83      1009
        1301       0.80      0.90      0.85       161
        1302       0.75      0.66      0.71       498
        1320       0.74      0.61      0.67       648
        1560       0.80      0.72      0.76      1015
        1920       0.89      0.85      0.87       861
        1940       0.49      0.90      0.63       161
        2060       0.7

# ============================================
# 7. (Optionnel) GridSearchCV sur quelques hyperparamètres
# ============================================

In [11]:
from sklearn.model_selection import GridSearchCV

# Petite grille de paramètres pour démonstration
param_grid = {
    "model__C": [0.5, 1.0, 2.0],
    "preprocess__title_tfidf__max_features": [10000, 20000],
    "preprocess__desc_tfidf__max_features": [20000, 30000],
}

grid = GridSearchCV(
    estimator=clf_pipeline,
    param_grid=param_grid,
    scoring="f1_weighted",
    cv=3,
    n_jobs=-1,
    verbose=2,
)

print("Lancement de la GridSearch (peut être long)...")
grid.fit(X_train, y_train)

print("Meilleurs paramètres trouvés :", grid.best_params_)
print("Meilleur score (F1 pondéré, CV) :", grid.best_score_)

# Évaluation du meilleur modèle sur le jeu de validation
best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_valid)
best_f1 = f1_score(y_valid, y_pred_best, average="weighted")
print(f"\nWeighted F1 du meilleur modèle sur validation : {best_f1:.4f}")
print("\nClassification report du meilleur modèle :")
print(classification_report(y_valid, y_pred_best))

Lancement de la GridSearch (peut être long)...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extr

[CV] END model__C=0.5, preprocess__desc_tfidf__max_features=20000, preprocess__title_tfidf__max_features=10000; total time=10.6min


/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END model__C=0.5, preprocess__desc_tfidf__max_features=20000, preprocess__title_tfidf__max_features=10000; total time=10.6min


...

[CV] END model__C=2.0, preprocess__desc_tfidf__max_features=30000, preprocess__title_tfidf__max_features=10000; total time= 5.3min


/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END model__C=2.0, preprocess__desc_tfidf__max_features=30000, preprocess__title_tfidf__max_features=20000; total time= 4.7min


/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Meilleurs paramètres trouvés : {'model__C': 2.0, 'preprocess__desc_tfidf__max_features': 20000, 'preprocess__title_tfidf__max_features': 10000}
Meilleur score (F1 pondéré, CV) : 0.7515288981620865

Weighted F1 du meilleur modèle sur validation : 0.7501

Classification report du meilleur modèle :
              precision    recall  f1-score   support

          10       0.32      0.64      0.43       623
          40       0.79      0.52      0.63       502
          50       0.78      0.75      0.77       336
          60       0.89      0.73      0.81       166
        1140       0.72      0.77      0.74       534
        1160       0.74      0.89      0.81       791
        1180       0.49      0.58      0.53       153
        1280       0.75      0.49      0.59       974
        1281       0.53      0.56      0.55       414
        1300       0.84      0.86      0.85      1009
        1301       0.81      0.91      0.86       161
        1302       0.76      0.67      0.71       498
